<a href="https://colab.research.google.com/github/noor848/python_ShAI/blob/main/Task_6_exercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![](logo1.jpg)

# **shAI Training 2021 | Level 1**

## Task #6 (End-to-End ML Project {part_2})

## Welcome to the exercises for reviewing second part of end to end ML project.
**Make sure that you read and understand ch2 from the hands-on ML book (page 72 to the end of the chapter ) before start with this notebook.**

**If you stuck with anything reread that part from the book and feel free to ask about anything in the messenger group as you go along.**

 ## Good Luck : )

## first run the following cell for the first part of the project to continue your work 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 
import seaborn as sns
from sklearn.model_selection import train_test_split
from pandas.plotting import scatter_matrix
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

In [2]:
import os
import tarfile
import urllib
DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml2/master/"
HOUSING_PATH = os.path.join("datasets", "housing")
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    os.makedirs(housing_path, exist_ok=True)
    tgz_path = os.path.join(housing_path, "housing.tgz")
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()
    
def load_housing_data(housing_path=HOUSING_PATH):
   csv_path = os.path.join(housing_path, "housing.csv")
   return pd.read_csv(csv_path)
   
fetch_housing_data()
housing = load_housing_data()

rooms_ix, bedrooms_ix, population_ix, household_ix = [
    list(housing.columns).index(col)
    for col in ("total_rooms", "total_bedrooms", "population", "households")]

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room = True):
        self.add_bedrooms_per_room = add_bedrooms_per_room
    def fit(self, X, y=None):
        return self  # nothing else to do
    def transform(self, X, y=None):
        rooms_per_household = X[:, rooms_ix] / X[:, household_ix]
        population_per_household = X[:, population_ix] / X[:, household_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household,
                         bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]
        
train_set, test_set = train_test_split(housing, test_size=0.2, random_state=42)
housing = train_set.drop("median_house_value", axis=1)
housing_labels = train_set["median_house_value"].copy()

housing_num = housing.drop("ocean_proximity", axis=1)
num_attribs = list(housing_num)
cat_attribs = ["ocean_proximity"]

num_pipeline = Pipeline([
 ('imputer', SimpleImputer(strategy="median")),
 ('attribs_adder', CombinedAttributesAdder()),
 ('std_scaler', StandardScaler())])

full_pipeline = ColumnTransformer([
 ("num", num_pipeline, num_attribs),
 ("cat", OneHotEncoder(), cat_attribs)])

housing_prepared = full_pipeline.fit_transform(housing) ####X

# 1- Select and Train a Model

# Let’s first train a LinearRegression model 

In [15]:
# CODE HERE

from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()



# First try it out on a few instances from the training set:


In [13]:
some_data = housing.iloc[:5]
some_labels = housing_labels.iloc[:5]

In [19]:
# CODE HERE

lin_reg.fit(housing_prepared,housing_labels)

some_data_prepared = full_pipeline.transform(some_data)
print("Predictions:", lin_reg.predict(some_data_prepared))
print( 'Labels:' , list(some_labels))


Predictions: [181746.54359616 290558.74973505 244957.50017771 146498.51061398
 163230.42393939]
Labels: [103000.0, 382100.0, 172600.0, 93400.0, 96500.0]


# measure this regression model’s RMSE on the whole training set 
* sing Scikit-Learn’s mean_squared_error() function:

In [ ]:
from sklearn.metrics import mean_squared_error

In [22]:
# CODE HERE


from sklearn.metrics import mean_squared_error
housing_predictlons = lin_reg.predict(housing_prepared) 
lin_nse = mean_squared_error(housing_labels, housing_predictlons) 
lin_nse = np.sqrt(lin_nse)
lin_nse



67593.20745775253

# judge on the RMSE result for this model 
write down your answar 

your answer goes here

# Let’s train a Decision Tree Regressor model 
## more powerful model

In [25]:
from sklearn.tree import DecisionTreeRegressor 

In [26]:
# CODE HERE
tree_reg = DecisionTreeRegressor()
tree_reg.fit(housing_prepared, housing_labels)




0.0

# Now evaluate the model on the training set 
* using Scikit-Learn’s mean_squared_error() function:

In [33]:
# CODE HERE

housing_predictlons = tree_reg.predict(housing_prepared) 
tree_mse = mean_squared_error(housing_labels, housing_predictlons)
tree_mse = np.sqrt(tree_mse) 
tree_mse




0.0

# Explaine this result 
write down your answar

your answer goes here

# Evaluation Using Cross-Validation

1-split the training set into 10 distinct subsets then train and evaluate the Decision Tree model

In [ ]:
from sklearn.model_selection import cross_val_score

In [30]:
# CODE HERE


from sklearn.model_selection import cross_val_score
scores = cross_val_score(tree_reg, housing_prepared, housing_labels,scoring="neg_mean_squared_error", cv=10) 
tree_rnse_scores = np.sqrt(-scores)
tree_rnse_scores


array([65546.97061046, 70901.56609   , 68883.74796111, 70194.25134169,
       72792.68434094, 67377.44494282, 66714.21840149, 67644.18227249,
       64595.48337524, 71005.93380083])

2- display the resultant scores and calculate its Mean and Standard deviation

In [35]:
# CODE HERE
def display_scores(scores): 
  print("Scores:", scores) 
  print( "Mean:", scores.mean())
  print("Standard deviation:", scores.std())
display_scores(tree_rnse_scores)


Scores: [65546.97061046 70901.56609    68883.74796111 70194.25134169
 72792.68434094 67377.44494282 66714.21840149 67644.18227249
 64595.48337524 71005.93380083]
Mean: 68565.6483137073
Standard deviation: 2501.923624943894


3-repaet the same steps to compute the same scores for the Linear Regression  model 

*notice the difference between the results of the two models*

In [37]:
# CODE HERE

scores = cross_val_score(lin_reg, housing_prepared, housing_labels,scoring="neg_mean_squared_error", cv=10) 
tree_rnse_scores = np.sqrt(-scores)
tree_rnse_scores

def display_scores(scores): 
  print("Scores:", scores) 
  print( "Mean:", scores.mean())
  print("Standard deviation:", scores.std())
display_scores(tree_rnse_scores)


Scores: [65000.67382615 70960.56056304 67122.63935124 66089.63153865
 68402.54686442 65266.34735288 65218.78174481 68525.46981754
 72739.87555996 68957.34111906]
Mean: 67828.38677377408
Standard deviation: 2468.091395065227


## Let’s train one last model the RandomForestRegressor.

In [ ]:
# CODE HERE

from sklearn.ensemble import RandomForestRegressor
forest_reg = RandomForestRegressor()
forest_reg.fit(housing_prepared, housing_labels)



# repeat the same steps to compute the same scores its Mean and Standard deviation for the Random Forest model

In [ ]:
# CODE HERE

scores = cross_val_score(forest_reg, housing_prepared, housing_labels,scoring="neg_mean_squared_error", cv=10) 
tree_rnse_scores = np.sqrt(-scores)
tree_rnse_scores

def display_scores(scores): 
  print("Scores:", scores) 
  print( "Mean:", scores.mean())
  print("Standard deviation:", scores.std())
display_scores(tree_rnse_scores)


# Save every model you experiment with 
*using the joblib library*

In [ ]:
# CODE HERE
import joblib
joblib.dump(forest_reg, "my_model.pkl")
my_model_loaded = joblib.load("my_model.pkl")


## now you have a shortlist of promising models. You now need to
## fine-tune them!
# Fine-Tune Your Model

## 1- Grid Search
## evaluate all the possible combinations of hyperparameter values for the RandomForestRegressor 
*It may take a long time*

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
# CODE HERE


from sklearn.model_selection import GridSearchCV
param_grid = [
{'n_estimators': [3, 16, 30], 'max_features': [2, 4, 6, 8]},
{'bootstrap': [False], 'n_estimators': [3, 10], 'nax_features': [2, 3, 4]},
]


with the evaluation scores

In [ ]:
# CODE HERE


forest_reg = RandomForestRegressor()
grid_search = GridSearchCV(forest_reg, param_grid, cv=5,
scoring='neg_nean_squared_error', return_train_score=True)
grid_search.fit(housing_prepared, housing_labels)


# Analyze the Best Models and Their Errors
1-indicate the relative importance of each attribute

In [ ]:
# CODE HERE


feature_importances = grid_search.best_estimator_.feature_importances.feature_importances_



2-display these importance scores next to their corresponding attribute names:

In [52]:
# CODE HERE


extra_attribs = ["roons_per_hhold" , "pop_per_hhold", "bedrooms_per_room"]

cat_encoder= full_pipeline.named_transformers_["cat"]
cat_one_hot_attribs = list(cat_encoder.categories_[0]) 
attributes = num_attribs + extra_attribs + cat_one_hot_attribs
sorted(zip(feature_importances, attributes), reverse=True)



NameError: ignored

## Now is the time to evaluate the final model on the test set.
# Evaluate Your System on the Test Set

1-get the predictors and the labels from your test set

In [ ]:
# CODE HERE

final_model = grid_search.best_estimator_
X_test = strat_test_set.drop( "median_house_value ", axis=l) 
y_test = strat_test_set[ "median_house_value "].copy()
X_test_prepared = full_pipeline.transform(X_test)



2-run your full_pipeline to transform the data

In [ ]:
# CODE HERE
final_predictions = final_model.predict(X_test_prepared)
final_mse = mean_squared_error(y_test, final_predictions)
final_rmse = np.sqrt(final_mse)



3-evaluate the final model on the test set

In [ ]:
# CODE HERE

final_predictions = final_model.predict(X_test_prepared)
final_mse = mean_squared_error(y_test, final_predictions)
final_rmse = np.sqrt(final_mse)



# compute a 95% confidence interval for the generalization error 
*using scipy.stats.t.interval():*

In [ ]:
from scipy import stats

In [ ]:
# CODE HERE

from scipy import stats 
confidence =0.95
squared_errors = (final_predictlons - y_test) ** 2
np.sqrt(stats.t.interval(confidence, len(squared_errors) -1,
loc=squared_errors.mean(), scale=stats.sem(squared_errors)))





# Great Job!
# #shAI_Club